In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import json
from pathlib import Path

In [3]:
path = Path('/data/google_nq/v1.0/train')
assert path.exists()

In [4]:
!ls -la {path}

total 89698580
drwxr-xr-x 2 root root       4096 Aug 25 19:30 .
drwxr-xr-x 5 root root       4096 Aug 25 16:07 ..
-rw-r--r-- 1 root root 5541577648 Aug 25 17:16 nq-train-00.jsonl
-rw-r--r-- 1 root root 5770348954 Aug 25 17:17 nq-train-01.jsonl
-rw-r--r-- 1 root root 5717125957 Aug 25 17:17 nq-train-02.jsonl
-rw-r--r-- 1 root root 5721671722 Aug 25 17:17 nq-train-03.jsonl
-rw-r--r-- 1 root root 5748361602 Aug 25 17:16 nq-train-04.jsonl
-rw-r--r-- 1 root root 5634916761 Aug 25 17:15 nq-train-05.jsonl
-rw-r--r-- 1 root root 5578705699 Aug 25 17:15 nq-train-06.jsonl
-rw-r--r-- 1 root root 5392278572 Aug 25 17:15 nq-train-07.jsonl
-rw-r--r-- 1 root root 5827270393 Aug 25 17:16 nq-train-08.jsonl
-rw-r--r-- 1 root root 5693204718 Aug 25 17:17 nq-train-09.jsonl
-rw-r--r-- 1 root root  876311133 Aug 25 17:17 nq-train-10.jsonl.gz
-rw-r--r-- 1 root root  878127326 Aug 25 17:16 nq-train-11.jsonl.gz
-rw-r--r-- 1 root root  889257016 Aug 25 17:17 nq-train-12.jsonl.gz
-rw-r--r-- 1 root root  89176912

In [5]:
dataset_files = [path/f for f in ['nq-train-00.jsonl', 'nq-train-01.jsonl', 'nq-train-02.jsonl', 'nq-train-03.jsonl', 'nq-train-04.jsonl', 'nq-train-05.jsonl', 'nq-train-06.jsonl', 'nq-train-07.jsonl', 'nq-train-08.jsonl', 'nq-train-09.jsonl']]
dataset_file = dataset_files[0]

In [6]:
!du -h {dataset_file}

5.2G	/data/google_nq/v1.0/train/nq-train-00.jsonl


In [7]:
!wc -l {dataset_file}

5961 /data/google_nq/v1.0/train/nq-train-00.jsonl


In [8]:
# !grep 'who has the best nba record in history' {dataset_file} > {path/'single_line.json'}

In [9]:
!head -n 1 {dataset_file} > {path/'single_line.json'}

In [10]:
with open(path/'single_line.json') as f:
    line = f.readline()

In [11]:
line_dict = json.loads(line)

In [12]:
line_dict.keys()

dict_keys(['annotations', 'document_html', 'document_title', 'document_tokens', 'document_url', 'example_id', 'long_answer_candidates', 'question_text', 'question_tokens'])

In [13]:
line_dict['question_text'], line_dict['document_title'], line_dict['question_tokens'][:5]

('when is the last episode of season 8 of the walking dead',
 'The Walking Dead (season 8)',
 ['when', 'is', 'the', 'last', 'episode'])

In [14]:
line_dict['annotations']

[{'annotation_id': 6782080525527814293,
  'long_answer': {'candidate_index': 92,
   'end_byte': 96948,
   'end_token': 3538,
   'start_byte': 82798,
   'start_token': 2114},
  'short_answers': [{'end_byte': 96731,
    'end_token': 3525,
    'start_byte': 96715,
    'start_token': 3521}],
  'yes_no_answer': 'NONE'}]

In [15]:
import string
sss = "this is a test, again"
"," in string.punctuation

True

In [16]:
AMOUNT_SENTENCES = 1

def shorten_context(context, answer):
    start_pos = context.find(answer)
    end_pos = context.find(answer) + len(answer)
    sentence_start = 0
    start_counter = 0
    for i in range(start_pos, -1, -1):
        if context[i] == '.':
            start_counter += 1
            if start_counter >= AMOUNT_SENTENCES:
                sentence_start = i + 1
                break
    sentence_end = len(context)
    end_counter = 0
    for i in range(end_pos + 1, sentence_end):
        if context[i] == '.':
            end_counter += 1
            if end_counter >= AMOUNT_SENTENCES:
                sentence_end = i
                break
    return context[sentence_start: sentence_end].strip()

In [17]:
def create_sentence(array):
#     res = ""
#     for s in reversed(array):
#         res =  (" " if len(s) > 1 or (s[0] not in string.punctuation or s[0] in ['(', ')', '-']) else "") + s + res
    sentence = ""
    for s in array:
        sentence += s + " "
    return sentence.strip()

def extract_short_answers(line_dict):
    answer_set = set([])
    question = line_dict['question_text'] + '?'
    context = create_sentence([t['token'] for t in line_dict['document_tokens'] if not t['html_token']])
    for anno in line_dict['annotations']:
        for sa in anno['short_answers']:
            tokens = [t for t in line_dict['document_tokens'][sa['start_token']:sa['end_token']] if not t['html_token']]
            answer = create_sentence([t['token'] for t in tokens])
            answer_set.add(answer)
    final_answers = []
    for a in answer_set:
        if context.find(a) > -1:
            short_context = shorten_context(context, a)
            if short_context.find(a) == -1:
                continue
            final_answers.append((question, a, {'answer_start': short_context.find(a), 'answer_end': short_context.find(a) + len(a)}, short_context))
    return final_answers
        
extract_short_answers(line_dict)

[('when is the last episode of season 8 of the walking dead?',
  'March 18 , 2018',
  {'answer_start': 193, 'answer_end': 208},
  "109 10 `` The Lost and the Plunderers '' TBA TBA March 4 , 2018 ( 2018 - 03 - 04 ) TBD 110 11 `` Dead or Alive Or '' TBA TBA March 11 , 2018 ( 2018 - 03 - 11 ) TBD 111 12 `` The Key '' TBA TBA March 18 , 2018 ( 2018 - 03 - 18 ) TBD Release ( edit ) The first trailer for the season was released on July 21 , 2017 , at San Diego Comic - Con")]

In [18]:
def navigate_data(full_data):
    answer_ds = []
    for i, l in enumerate(full_data):
        json_data = json.loads(l)
        for extracted in extract_short_answers(json_data):
            if len(extracted) > 0:
                answer_ds.append(extracted)
    return [a for a in answer_ds if len(a[1]) > 0]

In [19]:
def check_ds(answer_ds):
    count_not_found = []
    for i, a in enumerate(answer_ds):
        if a[2]['answer_start'] < 0:
            count_not_found.append((i, a))
        
    assert len(count_not_found) == 0

In [20]:
%%time

full_data = []
answer_datasets = []
for dataset_f in dataset_files:
    print(f'Processing {dataset_f}')
    with open(dataset_f) as f:
        full_data = f.readlines()
        answer_ds = navigate_data(full_data)
        check_ds(answer_ds)
        print(f'Extracted {len(answer_ds)} records out of {len(full_data)}')
        answer_datasets += answer_ds

Processing /data/google_nq/v1.0/train/nq-train-00.jsonl
Extracted 2559 records out of 5961
Processing /data/google_nq/v1.0/train/nq-train-01.jsonl
Extracted 2613 records out of 6138
Processing /data/google_nq/v1.0/train/nq-train-02.jsonl
Extracted 2658 records out of 6150
Processing /data/google_nq/v1.0/train/nq-train-03.jsonl
Extracted 2689 records out of 6160
Processing /data/google_nq/v1.0/train/nq-train-04.jsonl
Extracted 2658 records out of 6223
Processing /data/google_nq/v1.0/train/nq-train-05.jsonl
Extracted 2531 records out of 6152
Processing /data/google_nq/v1.0/train/nq-train-06.jsonl
Extracted 2581 records out of 6144
Processing /data/google_nq/v1.0/train/nq-train-07.jsonl
Extracted 2566 records out of 6074
Processing /data/google_nq/v1.0/train/nq-train-08.jsonl
Extracted 2700 records out of 6298
Processing /data/google_nq/v1.0/train/nq-train-09.jsonl
Extracted 2559 records out of 6177
CPU times: user 9min 9s, sys: 22 s, total: 9min 31s
Wall time: 10min 22s


In [21]:
len(answer_datasets)

26114

In [22]:
import pickle

pickle.dump( answer_datasets, open( "answers.pkl", "wb" ) )

In [23]:
!du -h answers.pkl

14M	answers.pkl
